In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!python3 -m pip install tensorflow

In [11]:
# 코드 작성일 : 24.7.4 -> 대표메뉴
import os
from pykospacing import Spacing
from concurrent.futures import ThreadPoolExecutor

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
exec(open('setup/default.py').read())

In [12]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [36]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    return text

# 데이터프레임에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)



In [14]:
first_menu_list = menu_w_sold_out.groupby(['first_menu'],dropna=False).size().to_frame('size').reset_index().sort_values(by='size',ascending=False)

In [17]:
# 띄어쓰기
spacing = Spacing()
sentences = first_menu_list.first_menu

def process_spacing(sentence):
    return spacing(sentence)

# ThreadPoolExecutor를 사용하여 동시에 5개의 작업을 수행합니다.
with ThreadPoolExecutor(max_workers=5) as executor:
    spaced_sentences = list(executor.map(process_spacing, sentences))

spaced_menu = pd.DataFrame({
    'first_menu': sentences,
    'first_menu_sp': spaced_sentences
})

In [32]:
#spaced_menu.to_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv',index=False)
spaced_menu=pd.read_csv('/Users/user/Desktop/haeyoon_private/cafeteria/data/feature_lab/menu_map/spaced_menu.csv')

In [37]:
menu_w_sold_out=pd.merge(menu_w_sold_out,spaced_menu, how='left',on='first_menu') 

In [38]:
first_menu_list = menu_w_sold_out.first_menu_sp.value_counts().to_frame().reset_index()

In [59]:
# 비교할 텍스트
target_text = '왕만두 떡국'

# 유사도 계산하여 가장 유사한 메뉴 찾기
first_menu_list['similarity'] = first_menu_list['first_menu_sp'].apply(lambda x: fuzz.token_sort_ratio(x, target_text))
first_menu_list.sort_values(by='similarity',ascending=False).iloc[:10,:]

,first_menu_sp,count,similarity
75,왕만두 떡국,6,100
249,사골 왕만두 떡국,2,80
67,떡 만두 국,6,67
358,사골 떡국,1,55
482,왕갈비 쌀국수,1,46
214,사골 떡만두국,2,46
513,교자 만두 칼국수,1,40
40,순대 국밥,8,36
25,장터 국밥,11,36
119,만두 전골,4,36
